In [3]:
import re

In [4]:
import pandas as pd

In [5]:
import csv

In [23]:
from word2number import w2n

In [181]:
# read csv containing Dengue articles to a dataframe
df = pd.read_csv('TestDengueContent.csv')

In [7]:
from mysql.connector import MySQLConnection, Error

In [8]:
config = {
  'user': 'root',
  'password': '',
  'host': '127.0.0.1',
  'database': 'dengue_sri_lanka',
  'raise_on_warnings': True
}

In [ ]:
import mysql.connector
from mysql.connector import Error
try:
#     mySQLconnection = mysql.connector.connect(host='localhost',
#                              database='dengue_sri_lanka',
#                              user='root',
#                              password='')
    conn = MySQLConnection(**config)
#     sql_select_Query = "select * from news_articles"
    select_new_articles = "select * from news_articles where isDataExtracted=0"

    cursor = conn .cursor()
    cursor.execute(select_new_articles)
    records = cursor.fetchall()
#     
    print("Total number of rows in news_articles is - ", cursor.rowcount)
    print ("Printing each row's column values i.e.  developer record")
    for row in records:
#        print("Id = ", row[0], )
#        print("Name = ", row[1])
#        print("JoiningDate  = ", row[2])
        print("Content  = ", row[3], "\n")
#     
    cursor.close()
   
except Error as e :
    print ("Error while connecting to MySQL", e)
finally:
    #closing database connection.
    if(conn .is_connected()):
        conn.close()
        print("MySQL connection is closed")


In [152]:
# Fill missing values
df = df.fillna("")

In [153]:
print(df.shape)

(20, 5)


In [154]:
# drops duplicate entries
df.drop_duplicates(subset=None, keep='first', inplace=True)

In [155]:
print(df.shape)

(20, 5)


In [137]:
regex_for_publication_date = re.compile(r"""(?:\d{4}-\d{2}-\d{2})|(?:\d{1,2}\s*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s*\d{4})""",re.IGNORECASE)

In [131]:
# Regex for publication date ####-##-##
regex_for_publication_date1 = r"\d{4}-\d{2}-\d{2}"

In [13]:
regex_for_numbers = re.compile(r'\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*');

In [96]:
regex_for_cases = re.compile(r'(\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d{1,3}){0,1})\s*(?:dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases|patients|dengue\spatients|from)', re.IGNORECASE)

In [ ]:
# a number following 'cases' or 'suspected dengue cases' or 'suspected cases'
regex_for_cases2 = re.compile(r'(\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*)\s*(?:dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases)', re.IGNORECASE)

In [14]:
regex_for_cases1 = re.compile(r'\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*\s*(?:dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases)', re.IGNORECASE)

In [63]:
regex_for_deaths = re.compile(r"""((?:\d+)|(?:[^\S]\d{1,3}(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)+)|
(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|
(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))
\s*
(?:deaths|death|dead\b|(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead\b)|dengue(?:\s|-)related\sdeath)
""", re.VERBOSE | re.IGNORECASE)

In [26]:
# regex for deaths or patients(TODO move patients to no.of cases)
regex_for_deaths1 = re.compile(r"""
(?:(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*)|
(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|
(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))
\s*
(?:deaths|death|dead\b|(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead\b)|patients|dengue\srelated\sdeath)

""", re.VERBOSE | re.IGNORECASE)

In [16]:
# regex to extract sentences having no. of cases/deaths
# TODO handle for "i.e" 
regex_for_sentences = re.compile(r"[^.]*?(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*\s*(?:dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases)|(?:(?:(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*)|(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))\s*(?:deaths|death|dead\b|(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead\b)|patients|dengue\srelated\sdeath)))[^.]*\.")

In [34]:
# regex for dates
regex_for_dates1 = re.compile(r"""(?:\d{4}(?:\.|/|-)\d{1,2}(?:\.|/|-)\d{1,2})|                                                                       # dates with format xxxx.xx.xx or xxxx/xx/xx or xxxx-xx-xx (month and date: 1 or 2 digits)
(?:\d{1,2}(?:\.|/|-)\d{1,2}(?:\.|/|-)\d{2,4})|                                                                                                      # dates with format xx.xx.xx where . or / or - used as the seperator, 1 or 2 digits for date or month, 2 or 4 digits for year
(?:(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s*\d{0,2})     # dates with format: month xx (date with 0-2 digits)
""", re.VERBOSE)

In [105]:
# regex_for_years 20xx
regex_for_years = re.compile(r"""\b20(?:\d{2}){1}\b|this\syear""", re.IGNORECASE)

In [36]:
# regex for months
regex_for_months = re.compile(r'Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)')

In [37]:
# regex for "so far"
regex_for_so_far = re.compile(r'so\sfar')

In [38]:
# regex for districts
regex_for_districts = re.compile(r'Jaffna|Kilinochchi|Mannar|Mullaitivu|Vavuniya|Puttalam|Kurunegala|Gampaha|Colombo|Kalutara|Anuradhapura|Polonnaruwa|Matale|Kandy|Nuwara\sEliya|Kegalle|Ratnapura|Trincomalee|Trinco|Batticaloa|Ampara|Badulla|Monaragala|Hambantota|Matara|Galle', re.IGNORECASE)

In [39]:
# regex for provinces
regex_for_provinces = re.compile(r'Western|Central|Eastern|North\sCentral|Northern|North\sWestern|Sabaragamuwa|Southern|Uva')

In [40]:
# regex for other places/locations
regex_for_places = re.compile(r'Sri Lanka|island|country|countrywide|Meethotamulla|Negombo|Ratmalana|Dangolla|Elpitiya|Beliatte|Hatton|Marakolliya|Gampola|Kalubowila')

In [144]:
get_published_date("2018-05-09 09:26:47 54 Thu, 07 Feb 2019 16:08:56 +0000")

['2018-05-09', '07 Feb 2019']

In [17]:
def get_sentences(text):
    return regex_for_sentences.findall(text)

In [145]:
regex_for_dates = re.compile(r"""(?:\d{1,2}\s*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|April|May|June|July|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s*\d{4})""",re.IGNORECASE)

In [177]:
def get_month_in_number(month):
    m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
        'may':5,
        'jun':6,
        'jul':7,
        'aug':8,
        'sep':9,
        'oct':10,
        'nov':11,
        'dec':12
        }
    s = month.strip()[:3].lower()

    try:
        out = m[s]
        return out
    except:
        raise ValueError('Not a month')
    

In [179]:
def get_published_date(text):
    dates = regex_for_publication_date.findall(text)
#     print(dates)
    for index,date in enumerate(dates):
        if(regex_for_dates.match(date)):
            day = re.search('\d{1,2}', date).group()
#             print(day)
#             print(re.search('\D{3,9}', date).group())
            month = get_month_in_number(re.search('\D{3,9}', date).group())
#             print(month)
            year = re.search('\d{4}', date).group()
#             print(year)
            date = year + "-" + str(month) + "-" + day
            dates[index] = date
    
    return dates

In [43]:
def get_numbers(text):
    return regex_for_numbers.findall(text)

In [98]:
def get_cases(text):
    cases = regex_for_cases.findall(text)
    for index,case in enumerate(cases):
        case = case.replace(" ","")
        case = case.replace(",","")
        case = int(case)   
        cases[index] = case
    return cases

In [45]:
def get_cases1(text):
    return regex_for_cases1.findall(text)

In [99]:
get_cases("345 cases and s 132, 999 cases reported yesterday, more than fifty deaths 12 deaths. Twenty six people have died")

[345, 132999]

In [80]:
regex_for_digits = re.compile(r'(?:(?:\s)*\d+(?:(?:\s)\d+)*)')

In [86]:
def get_deaths(text):
    deaths = regex_for_deaths.findall(text)
    print(deaths)
    for index,death in enumerate(deaths):
#         print(regex_for_digits.match(death))
        if(regex_for_digits.match(death)):
            death = death.replace(" ","")
            death = death.replace(",","")
#             print(death)
        death = w2n.word_to_num(death)   
        
        deaths[index] = death
    return deaths        

In [109]:
# ['2018']
def get_years(text,text_date):
    years = regex_for_years.findall(text)    
    for index,year in enumerate(years):
        if(year.lower() == "this year"):
            pub_date = get_published_date(text_date)
            year = re.search('\d{4}',pub_date).group()
            years[index] = year
    return years
            

In [112]:
def get_years1(text):
    return regex_for_years.findall(text)

In [48]:
def get_months(text):
    return regex_for_months.findall(text)

In [49]:
def get_dates(text):
    return regex_for_dates.findall(text)

In [50]:
def get_so_far(text):
    return regex_for_so_far.findall(text)

In [51]:
def get_provinces(text):
    return regex_for_provinces.findall(text)

In [52]:
def get_districts(text):
    return regex_for_districts.findall(text)

In [53]:
def get_places(text):
    return regex_for_places.findall(text)

In [ ]:
locations=["Meethotamulla","Colombo", "kandy","Anuradhapura","Galle","Matara","Ratmalana","Dangolla","Badulla","Elpitiya","Beliatte","Hatton","Marakolliya","Gampola","Kalubowila","Puttalam",]

In [54]:
articles=[]

In [55]:
regex_for_sentences = re.compile(r"[^.]*?(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*\s*(?:dengue|dengue\scases|dengue\sfever\scases|cases|suspected\sdengue\scases|suspected\scases|suspected\sdengue\sfever\scases)|(?:(?:(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*)|(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))\s*(?:deaths|death|dead\b|(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead\b)|patients|dengue\srelated\sdeath)))[^.]*\.")

In [ ]:
regex_for_sentences1 = re.compile(r"[^.]*?(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*\s*(?:dengue|cases|suspected\sdengue\scases|suspected\scases)|(?:(?:(?:\d+(?:(?:,|\s|(?:,\s)|(?:\s,))\d+)*)|(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|(?:(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)\s*)(?:one|two|three|four|five|six|seven|eight|nine)?)))\s*(?:deaths|death|dead(?:people\s*|patients\s*)?(?:have\s*|had\s*)?(?:died|dead)|patients)))[^.]*\.")

In [180]:
for index, row in df.iterrows():
#     print(regex_for_sentences.findall(row['Article_content']))
#     print("*")
#     for sentence in regex_for_sentences.findall(row['Article_content']):
#         print(regex_for_sentences.findall(row['Article_content']))
#         print(get_deaths1(sentence))
    print(get_published_date(row['Date']))
#         print(get_years(sentence,row['Date']))
#         print(get_years1(sentence))
#     print()
    

['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-17']
['2019-1-16']
['2019-1-16']
['2019-1-16']
['2019-1-16']
['2019-1-16']
['2019-1-16']


In [ ]:
articles=[]
for index, row in df.iterrows():
    print("Index: "+str(index))
#     print("Publication date: "+get_published_date(row['Date']))
    sentences = get_sentences(row['Article_content'])
#     print(sentences)
    for sentence in sentences:
        print(sentence)
#         get_data(sentence)
        
        articles.append({
                'Index':str(index),
                'Publication_date':get_published_date(row['Date']),
                'Sentence':sentence.encode("utf-8"),
                'Numbers':get_cases1(sentence),
                'Cases':get_cases(sentence),
                'Deaths':get_deaths(sentence),
                'Years':get_years(sentence),
                'Months':get_months(sentence),
                'Dates': get_dates(sentence),
                'Time_duration':get_so_far(sentence),
                'Provinces':get_provinces(sentence),
                'Districts':get_districts(sentence),
                'Places':get_places(sentence),
            })
        
#     print()

In [ ]:
articles=[]
for row in records:
    sentences = get_sentences(row[3])
    for sentence in sentences:
        print(sentence)
        
        articles.append({
                'Index':str(index),
                'Publication_date':get_published_date(row[1]),
                'Sentence':sentence.encode("utf-8"),
                'Numbers':get_cases1(sentence),
                'Cases':get_cases(sentence),
                'Deaths':get_deaths(sentence),
                'Years':get_years(sentence),
                'Months':get_months(sentence),
                'Dates': get_dates(sentence),
                'Time_duration':get_so_far(sentence),
                'Provinces':get_provinces(sentence),
                'Districts':get_districts(sentence),
                'Places':get_places(sentence),
            })
        
#     print()

In [ ]:
articles=[]
conn = MySQLConnection(**config)
cursor = conn.cursor()

for row in records:
#     print(row[3].decode("utf-8"))
    sentences = get_sentences(row[3].decode("utf-8"))
#     print(sentences)
    for sentence in sentences:
        print(sentence)
        
        publication_date = get_published_date(row[1])
#         sentence = sentence.encode("utf-8")
#         numbers = get_cases1(sentence)
        cases = get_cases(sentence)
        deaths = get_deaths(sentence)
        year = get_years(sentence)
        month = get_months(sentence)
#         dates = get_dates(sentence)
        time_duration = get_so_far(sentence)
        province = get_provinces(sentence)
        district = get_districts(sentence)
        countrywide = get_places(sentence)
        
        query = "INSERT IGNORE INTO regex_output(publication_date, cases, deaths, year, month, time_duration, province, countrywide, article_id, district) " \
            "VALUES(publication_date, cases, deaths, year, month, time_duration, province, countrywide, 1, district)"
        
        try:
#         db_config = read_db_config()
 
            cursor.execute(query)
            conn.commit()
            
        except Error as error:
            print(error)
        
cursor.close()
conn.close()

In [ ]:
articles=[]
conn = MySQLConnection(**config)
cursor = conn.cursor()

# for row in records:
#     print(row[3].decode("utf-8"))
#     sentences = get_sentences(row[3].decode("utf-8"))
#     print(sentences)
#     for sentence in sentences:
#         print(sentence)
        
#         publication_date = get_published_date(row[1])
# #         sentence = sentence.encode("utf-8")
# #         numbers = get_cases1(sentence)
#         cases = get_cases(sentence)
#         deaths = get_deaths(sentence)
#         year = get_years(sentence)
#         month = get_months(sentence)
# #         dates = get_dates(sentence)
#         time_duration = get_so_far(sentence)
#         province = get_provinces(sentence)
#         district = get_districts(sentence)
#         countrywide = get_places(sentence)
        
query = "INSERT IGNORE INTO regex_output(publication_date, cases, deaths, year, month, time_duration, province, countrywide, article_id, district) " \
            "VALUES('01/01/2018', 23455, 234, ['2019'], 'month', 'time', 'province', 'countrywide', 1, 'district')"
        
try:
#         db_config = read_db_config()
 
    cursor.execute(query)
    conn.commit()
            
except Error as error:
    print(error)
        
cursor.close()
conn.close()

In [ ]:
# TODO IMPORTANT: set the isDataExtracted column of each row to 1

In [ ]:
# write data in the 'article' list to a csv
with open('Regex_output_test1.csv',mode='w') as csv_file:
    fieldnames=['Index','Publication_date','Sentence','Numbers','Cases','Deaths','Years','Months','Dates','Time_duration','Provinces','Districts','Places']
    writer=csv.DictWriter(csv_file,fieldnames=fieldnames)

    writer.writeheader()
    for article in articles:
        writer.writerow({'Index':article['Index'],'Publication_date':article['Publication_date'],'Sentence':article['Sentence'],'Numbers':article['Numbers'],'Cases':article['Cases'],'Deaths':article['Deaths'],'Years':article['Years'],'Months':article['Months'],'Dates':article['Dates'],'Time_duration':article['Time_duration'],'Provinces':article['Provinces'],'Districts':article['Districts'],'Places':article['Places']})


In [ ]:
# import re

# locations=["Meethotamulla","Colombo", "kandy","Anuradhapura","Galle","Matara","Ratmalana","Dangolla","Badulla","Elpitiya","Beliatte","Hatton","Marakolliya","Gampola","Kalubowila","Puttalam",]

# import pandas as pd
# import csv

# df = pd.read_csv('TestDengueContent.csv')
# df = df.fillna("")
# print(df.shape)

# for index, row in df.iterrows():
#         for line in (row):
#             i=0
#             while i<len(locations):
#                 if locations[i] in line:
#                     print(locations[i])
#                 i+=1

# # for index, row in df.iterrows():
# #         for line in (row):
# #             for location in locations:
# #                 if location in line:
# #                     print(location)


In [ ]:
import nltk

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
data = "Contamination of the food chain and waterways continue to pose serious public health risks in many countries including Sri Lanka.		Moreover, the spread of unhealthy food habits, often propagated by big multinationals, has become a public health menace not just in developing countries but in the most developed parts of the world as well, i.e. America, Europe, Australasia and elsewhere.		Recognizing the wider background to growing public health risks in different parts of the world, the WHO commissioned and publishedï¿½ several years ago a well-articulated report on Social Determinants of Health (SDH) . "


In [ ]:
print(sent_tokenize(data))

In [ ]:
# word to number

In [51]:
from word2number import w2n

In [53]:
print(w2n.word_to_num('Twenty six'))

26


In [120]:
import calendar

In [126]:
list(calendar.month_abbr).index('Jan')

1